# Chapter 10 Data Aggregation and Group Operations

- Split a data frame into pieces using one or more keys.
- Calculate group summary statistics such as count, mean, standard deviation, or a user-defined function.
- Apply within-group transformations such as normalization.
- Compute pivot tables and cross-tabulations.
- Perform statistical group analysis.

## I. GroupBy Mechanics

Many data processing follows a **split-apply-combine** process. For example, you may want to do the following operations to analyze a dataset about sales:
1. What is the total revenue every day?
2. What is the total sales of each product?
3. How much has each client perchased in total?

These operations all requires that you split the data into groups, and then apply certain calculations to each of the groups, and finally combine all results into a new table. In Pandas this is mostly done with `groupby()` function.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# An example:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.451789,0.289296
1,a,two,0.367263,0.855379
2,b,one,0.800570,-0.740288
3,b,two,0.355643,-1.414836
4,a,one,-0.403022,-0.572666


In [3]:
# Split data1 values according to key1:
groups = df['data1'].groupby(df['key1'])
groups

In [4]:
# Apply mean() function to find the average value for each group
means = groups.mean()
means

key1
a   -0.162516
b    0.578107
Name: data1, dtype: float64

In [5]:
# Convert it to a data frame
df_means = means.to_frame(name='data1_mean')
df_means

,data1_mean
key1,
a,-0.162516
b,0.578107


In [6]:
# Put all operations in one statement
df_means = df['data1'].groupby(df['key1']).mean().to_frame(name='data1_mean')
df_means

,data1_mean
key1,
a,-0.162516
b,0.578107


In [7]:
# Exercise: split data2 according to key2, and calculate the sum.
df_sums = df['data2'].groupby(df['key2']).sum().to_frame(name = "data2 sum")
df_sums

,data2 sum
key2,
one,-1.023657
two,-0.559457


We can use more than one column as keys.

In [8]:
# Split the data according to both key1 and key2
groups = df['data1'].groupby([df['key1'], df['key2']])

In [9]:
# Calculate the mean
means = groups.mean()
means

key1  key2
a     one    -0.427406
      two     0.367263
b     one     0.800570
      two     0.355643
Name: data1, dtype: float64

We obtain a pandas Series with **hierarchical indexing**. It can be converted to a data frame using `unstack()`.

In [10]:
# Convert it to a data frame
means.unstack()

key2,one,two
key1,,
a,-0.427406,0.367263
b,0.800570,0.355643


In [11]:
# Put all operations in one statement
df["data1"].groupby([df["key1"], df["key2"]]).mean().unstack()


key2,one,two
key1,,
a,-0.427406,0.367263
b,0.800570,0.355643


In [12]:
# Split the entire data frame
df.groupby([df['key1'], df['key2']]).mean()

data1     data2
key1 key2                    
a    one  -0.427406 -0.141685
     two   0.367263  0.855379
b    one   0.800570 -0.740288
     two   0.355643 -1.414836

In [13]:
# Frequently the grouping information is found in the same data frame as the data 
# you want to work on. In that case, simply put column names as the keys:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.427406 -0.141685
     two   0.367263  0.855379
b    one   0.800570 -0.740288
     two   0.355643 -1.414836

In [14]:
# Find the number of instances in each subgroup
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

**Iterating Over Groups**

The GroupBy object support iteration, providing a sequence of 2-tuples containing the group name along with the data.

In [15]:
# Show the content of each group.
groups = df.groupby(['key1', 'key2'])
for name, group in groups:
    print("Name:", name)
    print(group)

Name: ('a', 'one')
  key1 key2     data1     data2
0    a  one -0.451789  0.289296
4    a  one -0.403022 -0.572666
Name: ('a', 'two')
  key1 key2     data1     data2
1    a  two  0.367263  0.855379
Name: ('b', 'one')
  key1 key2    data1     data2
2    b  one  0.80057 -0.740288
Name: ('b', 'two')
  key1 key2     data1     data2
3    b  two  0.355643 -1.414836


**Syntactic sugar**: selecting columns for groupby()

In [16]:
df.groupby('key1')['data1'].min()

key1
a   -0.451789
b    0.355643
Name: data1, dtype: float64

In [17]:
df['data1'].groupby(df['key1']).min()

key1
a   -0.451789
b    0.355643
Name: data1, dtype: float64

In [18]:
df.groupby('key1')[['data2']].min()

,data2
key1,
a,-0.572666
b,-1.414836


In [19]:
df[['data2']].groupby(df['key1']).min()

,data2
key1,
a,-0.572666
b,-1.414836


**Grouping with dictionary**

In [20]:
values = np.array([
    [100, 80, 95],
    [55, 60, 45],
    [70, 75, 90],
    [75, 70, 60],
    [60, 73, 75],
    [72, 63, 70]
])
data = pd.DataFrame(values,
                   columns=['Midterm', 'Project', 'Final'],
                   index=['Alics', 'Bob', 'Chris', 'Doug', 'Eva', "Frank"])
data

,Midterm,Project,Final
Alics,100,80,95
Bob,55,60,45
Chris,70,75,90
Doug,75,70,60
Eva,60,73,75
Frank,72,63,70


In [21]:
gender = {
    'Alics': 'F',
    'Bob': 'M',
    'Chris': 'M',
    'Doug': 'M',
    'Eva': 'F',
    'Frank': 'M'
}

In [22]:
# split the rows according to gender
data.groupby(gender).size()

F    2
M    4
dtype: int64

In [23]:
# same as grouping with lists
data.groupby(["F", "M", "M", "M", "F", "M"]).size() # not recommended

F    2
M    4
dtype: int64

In [24]:
data.groupby(gender).mean()

,Midterm,Project,Final
F,80.0,76.5,85.00
M,68.0,67.0,66.25


**Grouping with functions**

Any function passed as a group key will be called once per index value, with the returned values being used as the group names.

In [25]:
def get_initial(name):
    return name[0]

In [26]:
data.groupby(get_initial).mean()

,Midterm,Project,Final
A,100,80,95
B,55,60,45
C,70,75,90
D,75,70,60
E,60,73,75
F,72,63,70


In [27]:
data.groupby(lambda x: x[0]).mean()

,Midterm,Project,Final
A,100,80,95
B,55,60,45
C,70,75,90
D,75,70,60
E,60,73,75
F,72,63,70


**Example: Filling Missing Values with Group-Specific Values**

In [28]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.DataFrame(np.random.randn(8), index=states, columns=['Value'])
data.loc[['Vermont', 'Nevada', 'Idaho']] = np.nan
data['group_key'] = group_key
data

,Value,group_key
Ohio,0.374525,East
New York,0.210705,East
Vermont,NaN,East
Florida,0.689503,East
Oregon,1.028816,West
Nevada,NaN,West
California,-0.594836,West
Idaho,NaN,West


In [29]:
# Fill the missing values with mean value
data.fillna(data.mean())


,Value,group_key
Ohio,0.374525,East
New York,0.210705,East
Vermont,0.341743,East
Florida,0.689503,East
Oregon,1.028816,West
Nevada,0.341743,West
California,-0.594836,West
Idaho,0.341743,West


In [30]:
# Find the average value of eastern states and western states
means = data.groupby("group_key").mean()

# Fill missing values with group specific average
#data.groupby("group_key").apply(lambda x: x.fillna(x.mean()))

def fill_group(group):
    return group.fillna(group.mean())
data.groupby("group_key").apply(fill_group)

Value group_key
group_key                               
East      Ohio        0.374525      East
          New York    0.210705      East
          Vermont     0.424911      East
          Florida     0.689503      East
West      Oregon      1.028816      West
          Nevada      0.216990      West
          California -0.594836      West
          Idaho       0.216990      West

In [31]:
# Fill missing values with the following rule:
# East: 0.5
# West: -0.5
values = {"East": 0.5,
          "West": -0.5}
#data.groupby("group_key").apply(lambda x: x.fillna(values[x.name]))

def fill_group2(group):
    value = values[group.name]
    return group.fillna(value)
data.groupby("group_key").apply(fill_group2)

,Value,group_key
Ohio,0.374525,East
New York,0.210705,East
Vermont,0.500000,East
Florida,0.689503,East
Oregon,1.028816,West
Nevada,-0.500000,West
California,-0.594836,West
Idaho,-0.500000,West


**Example: Random Sampling and Permutation**

In [32]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
QS     10
KS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
QC     10
KC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
QD     10
KD     10
dtype: int64

In [33]:
# Randomly sample 5 rows
deck.sample(5)


8S     8
6C     6
KD    10
6D     6
4S     4
dtype: int64

In [34]:
# Randomly sample 2 cards from each suit
groups = deck.groupby(lambda x: x[-1]) # takes the last character of the index 

#for name, group in groups:
#   print(name)
#  print(group)

groups.apply(lambda x: x.sample(2))

C  10C    10
   7C      7
D  10D    10
   8D      8
H  5H      5
   QH     10
S  4S      4
   10S    10
dtype: int64

**Example: Analyzing Cell Phone History**

In [35]:
# Load data
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
url = "https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2015/06/phone_data.csv"
# data = pd.read_csv(url, delimiter=",")
data = pd.read_csv(url, delimiter=",", index_col='index')
print(data.shape)
data.head(3)

(830, 6)


,date,duration,item,month,network,network_type
index,,,,,,
0,15/10/14 06:58,34.429,data,2014-11,data,data
1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile


1. **date**: The date and time of the entry
2. **duration**: The duration (in seconds) for each call, the amount of data (in MB) for each data entry, and the number of texts sent (usually 1) for each sms entry.
3. **item**: A description of the event occurring – can be one of call, sms, or data.
4. **month**: The billing month that each entry belongs to – of form ‘YYYY-MM’.
5. **network**: The mobile network that was called/texted for each entry.
6. **network_type**: Whether the number being called was a mobile, international (‘world’), voicemail, landline, or other (‘special’) number.

In [36]:
# Convert date column from string to datetime objects
from dateutil.parser import parse
data['date'] = data['date'].apply(parse, dayfirst=True)
data.head(3)

,date,duration,item,month,network,network_type
index,,,,,,
0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile


In [37]:
# Check data types
data.dtypes


date            datetime64[ns]
duration               float64
item                    object
month                   object
network                 object
network_type            object
dtype: object

In [38]:
# Convert date column from string to datetime objects
from dateutil.parser import parse
data["date"] = data["date"].apply(parse, dayfirst = True)
data.head(3)

TypeError: Parser must be a string or character stream, not Timestamp

In [39]:
# Check missing values
data.isnull().sum()


date            0
duration        0
item            0
month           0
network         0
network_type    0
dtype: int64

**Apply GroupBy actions**

In [40]:
# Which months are covered in this data set?
data.groupby(['month']).groups.keys()

dict_keys(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'])

In [41]:
# Find the first entry for each month
data.groupby(['month']).first()

,date,duration,item,network,network_type
month,,,,,
2014-11,2014-10-15 06:58:00,34.429,data,data,data
2014-12,2014-11-13 06:58:00,34.429,data,data,data
2015-01,2014-12-13 06:58:00,34.429,data,data,data
2015-02,2015-01-13 06:58:00,34.429,data,data,data
2015-03,2015-02-12 20:15:00,69.000,call,landline,landline


In [42]:
# Get the number of instances in each month
data["month"].value_counts()


2014-11    230
2015-01    205
2014-12    157
2015-02    137
2015-03    101
Name: month, dtype: int64

In [43]:
# What is the sum of call durations for each month?
data.groupby("month").sum()

,duration
month,
2014-11,26639.441
2014-12,14641.870
2015-01,18223.299
2015-02,15522.299
2015-03,22750.441


**Group by more than one variable**

In [44]:
# How many calls, messages, and data entries are there in each month?
data.groupby(["month", "item"]).count()


date  duration  network  network_type
month   item                                       
2014-11 call   107       107      107           107
        data    29        29       29            29
        sms     94        94       94            94
2014-12 call    79        79       79            79
        data    30        30       30            30
        sms     48        48       48            48
2015-01 call    88        88       88            88
        data    31        31       31            31
        sms     86        86       86            86
2015-02 call    67        67       67            67
        data    31        31       31            31
        sms     39        39       39            39
2015-03 call    47        47       47            47
        data    29        29       29            29
        sms     25        25       25            25

In [45]:
# How many instances are there per month, split by network_type?
data.groupby(["month", "network_type"]).count()


date  duration  item  network
month   network_type                               
2014-11 data            29        29    29       29
        landline         5         5     5        5
        mobile         189       189   189      189
        special          1         1     1        1
        voicemail        6         6     6        6
2014-12 data            30        30    30       30
        landline         7         7     7        7
        mobile         108       108   108      108
        voicemail        8         8     8        8
        world            4         4     4        4
2015-01 data            31        31    31       31
        landline        11        11    11       11
        mobile         160       160   160      160
        voicemail        3         3     3        3
2015-02 data            31        31    31       31
        landline         8         8     8        8
        mobile          90        90    90       90
        special          2         2     2        2
        voicemail        6         6     6        6
2015-03 data            29        29    29       29
        landline        11        11    11       11
        mobile          54        54    54       54
        voicemail        4         4     4        4
        world            3         3     3        3

## II. Data Aggregation
Aggregation refer to any data transformation that produces numeric values from arrays. The preceding examples have used several of them, including `mean()`, `count()`, `first()`, `min()`, and `sum()`. However, user-defined functions can also be applied to create desired summary.

In [46]:
# Define function range() that returns(max - min)
def get_range(array):
    return array.max() - array.min()

In [47]:
# Apply agg() to find the range of each type of cell phone use.
data.groupby(['item'])['duration'].agg(get_range)

item
call    10527.0
data        0.0
sms         0.0
Name: duration, dtype: float64

In [48]:
# Apply multiple aggregation functions
data.groupby(['item'])['duration'].agg([get_range, np.max, np.min])

,get_range,amax,amin
item,,,
call,10527.0,10528.000,1.000
data,0.0,34.429,34.429
sms,0.0,1.000,1.000


In [49]:
# Declare columns names
data.groupby(['item'])['duration'].agg([('range', get_range),
                                        ('maximum', np.max),
                                        ('minimum', np.min)])

,range,maximum,minimum
item,,,
call,10527.0,10528.000,1.000
data,0.0,34.429,34.429
sms,0.0,1.000,1.000


In [50]:
# Apply a different function to each column
functions = {
    'duration': sum,
    'network_type': 'count',
    'date': 'first'
}
data.groupby(['month', 'item']).agg(functions)

duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

In [51]:
# Tuple named aggregations
data[data['item'] == 'call'].groupby('month').agg(
    # Get max of the duration column for each group
    max_duration=('duration', max),
    # Get min of the duration column for each group
    min_duration=('duration', min),
    # Get sum of the duration column for each group
    total_duration=('duration', sum),
    # Apply a lambda to date column
    num_days=("date", lambda x: (max(x) - min(x)).days)    
)

TypeError: aggregate() missing 1 required positional argument: 'arg'

## III. Pivot Table
It is used to split the data using two sets of keys. 


In [53]:
# Create a pivot table with counts for each month and network type
data.pivot_table("date", index = "month", columns = "network_type", aggfunc = len)

network_type,data,landline,mobile,special,voicemail,world
month,,,,,,
2014-11,29.0,5.0,189.0,1.0,6.0,NaN
2014-12,30.0,7.0,108.0,NaN,8.0,4.0
2015-01,31.0,11.0,160.0,NaN,3.0,NaN
2015-02,31.0,8.0,90.0,2.0,6.0,NaN
2015-03,29.0,11.0,54.0,NaN,4.0,3.0


## IV. Cross Tabulation

In [56]:
pd.crosstab(index = data["month"], columns = data["network_type"])

network_type,data,landline,mobile,special,voicemail,world
month,,,,,,
2014-11,29,5,189,1,6,0
2014-12,30,7,108,0,8,4
2015-01,31,11,160,0,3,0
2015-02,31,8,90,2,6,0
2015-03,29,11,54,0,4,3


# Homework:
Use the cell phone usage data in this exercise.
1. Find out the network names that belongs to network_type "mobile".
2. How many messages were sent to each mobile network every month?
3. What is the total call duration to each mobile network every month?

In [ ]:
data.groupby(['network_type', 'network']).count()

In [ ]:
subdata = data[data['network_type'] == 'mobile']
subdata['network'].value_counts()

In [ ]:
subdata.groupby(['month', 'network']).count()

In [ ]:
subdata[subdata['item'] == 'call'].groupby(['month', 'network'])['duration'].sum()